In [1]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import os

# PASCAL VOC 형식에서 YOLO 형식으로 변환하는 함수
def voc_to_yolo(x_min, y_min, x_max, y_max, img_width, img_height):
    # 중심 좌표 계산
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    
    # 너비 및 높이 계산
    width = x_max - x_min
    height = y_max - y_min
    
    # 이미지 크기에 대한 상대값으로 변환
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height
    
    return x_center, y_center, width, height

# Pascal VOC 형식을 변환하는 함수
def convert_voc_to_yolo(txt_file_path, image_dir, labels_dir):
    # labels 폴더가 존재하지 않으면 생성
    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir)

    with open(txt_file_path, "r") as file:
        lines = file.readlines()

    for line in lines:
        # 각 줄에서 정보 추출
        image_name, x_min, y_min, x_max, y_max, class_id = line.strip().split()
        
        # 이미지 파일 경로 생성
        img_path = os.path.join(image_dir, image_name)
        
        # 이미지 크기 읽기 (OpenCV 사용)
        img = cv2.imread(img_path)
        if img is None:
            print(f"이미지를 읽을 수 없습니다: {img_path}")
            continue

        img_height, img_width, _ = img.shape
        
        # PASCAL VOC 형식에서 YOLO 형식으로 변환
        x_min, y_min, x_max, y_max = map(int, [x_min, y_min, x_max, y_max])
        x_center, y_center, width, height = voc_to_yolo(x_min, y_min, x_max, y_max, img_width, img_height)
        
        # YOLO 형식: class_id x_center y_center width height
        # 해당 이미지의 이름을 기준으로 .txt 파일 생성
        label_file_path = os.path.join(labels_dir, image_name.replace(".jpg", ".txt"))
        with open(label_file_path, "w") as label_file:
            label_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# 경로 설정 (txt 파일 경로, 이미지 디렉토리 경로, 결과를 저장할 파일 경로)
txt_file_path = "rec.txt"  # PASCAL VOC 형식이 적힌 txt 파일 경로
image_dir = "rec"               # 원본 이미지가 있는 디렉토리
# output_file_path = "yolo.txt"  # YOLO 형식으로 변환된 결과 파일 경로

# 경로 설정 (txt 파일 경로, 이미지 디렉토리 경로, labels 폴더 경로)
# txt_file_path = "path/to/your/annotations.txt"  # PASCAL VOC 형식이 적힌 txt 파일 경로
# image_dir = "path/to/your/images"               # 원본 이미지가 있는 디렉토리
labels_dir = "labels"              # YOLO 형식으로 변환된 파일을 저장할 labels 폴더

# 변환 실행
convert_voc_to_yolo(txt_file_path, image_dir, labels_dir)
